In [1]:
!pip install -q transformers accelerate bitsandbytes fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00


In [2]:
import nest_asyncio
from pyngrok import ngrok
nest_asyncio.apply()

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from fastapi.middleware.cors import CORSMiddleware
app = FastAPI()

In [4]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
system_message = """Tu es l'assistant virtuel officiel de l'École Nationale d'Intelligence Artificielle et du Digital (ENIAD) de Berkane, Maroc.

Informations sur l'ENIAD Berkane:
- Établissement d'excellence dédié à la formation en IA et technologies digitales
- Situé à Berkane, Maroc
- Offre des programmes en Licence en IA et Science des Données, Master en Ingénierie des Systèmes Intelligents
- Propose des formations continues en Digital Marketing et des certifications en Développement Web et Mobile
- L'école vise à former les futurs experts en technologies numériques et intelligence artificielle

INSTRUCTIONS DE FORMATAGE MARKDOWN STRICTES:
1. Utilise des titres avec # et ## pour structurer ta réponse
2. Utilise des listes à puces avec - pour énumérer des points
3. Mets en **gras** les informations importantes
4. Utilise des *italiques* pour l'emphase
5. Utilise des sauts de ligne entre les paragraphes
6. Utilise des > pour les citations ou informations importantes

Exemple de structure:
# Réponse à votre question

## Présentation générale
Voici un paragraphe d'introduction sur le sujet.

## Points importants
- Premier point important
- Deuxième point avec **information clé**
- Troisième point à retenir

## Détails supplémentaires
Voici quelques détails additionnels sur le sujet.

> Note importante: information à ne pas oublier

Ton objectif est de fournir des informations précises, bien structurées et faciles à lire en utilisant correctement la syntaxe Markdown.
"""


In [6]:
repo_id = "ahmed-ouka/my-llama3.1-8B-with-lora-Eniad-Assistant"
tokenizer = AutoTokenizer.from_pretrained(repo_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
import torch
torch.cuda.empty_cache()

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    quantization_config=bnb_config,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors.index.json:   0%|          | 0.00/61.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [9]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

In [10]:
class Message(BaseModel):
    messages: list

@app.post("/generate")
async def generate_response(message: Message):
    trigger_msg = {
        "role": "user",
        "content": (
            "Merci d'appliquer STRICTEMENT le format : "
            "réponse directe uniquement. "
            "**Et surtout, réponds toujours dans la même langue que la dernière question!!!.**"
        )
    }

    full_messages = [{"role": "system", "content": system_message}, trigger_msg] + message.messages

    try:
        prompt = tokenizer.apply_chat_template(full_messages, tokenize=False, add_generation_prompt=True)
    except:
        prompt = "".join(f"<{m['role']}>{m['content']}\n" for m in full_messages)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.3,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )

    gen_ids = output[0][inputs.input_ids.shape[-1]:]
    reply = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    return {"reply": reply}

In [11]:
# Configurez votre token ngrok ici
!ngrok config add-authtoken 2xHno0eEIibojVzcWMINdkV2PsQ_38Vdnh8oU8R1JpwiqzBzm
import uvicorn

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
public_url = ngrok.connect(8000)
print(f"🔗(POST): {public_url}/generate")

🔗(POST): NgrokTunnel: "https://27bb-35-199-163-41.ngrok-free.app" -> "http://localhost:8000"/generate


In [13]:
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [770]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     196.70.137.4:0 - "OPTIONS /generate HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


INFO:     196.70.137.4:0 - "POST /generate HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


INFO:     196.70.137.4:0 - "POST /generate HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


INFO:     196.70.137.4:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [770]
